In [0]:
file_location = "s3://consumer-orbit-us-east-1-holding/Verisk/TCI/*"
df_data = spark.read.option("delimiter", "\t").option("inferSchema", "False").csv(file_location, header=False)
display(df_data)

In [0]:
# File location and type #
# catalog location : 's3://consumer-orbit-us-east-1-holding/Total_Consumer_Insights_Plus_Counts (5) (1).xlsx'

file_location2 = 's3://consumer-orbit-us-east-1-holding/Total_Consumer_Insights_Plus_Counts (5) (1).xlsx'
file_type = "com.crealytics.spark.excel"

Total_Consumer_Insights_Counts_df = spark.read.format(file_type) \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .option("dataAddress", "'Sheet1'!A11") \
  .load(file_location2)

# Drop rows where all columns are null
Total_Consumer_Insights_Counts_df = Total_Consumer_Insights_Counts_df.dropna(how='all')

# Display the schema to find the correct column name
Total_Consumer_Insights_Counts_df.printSchema()


display(Total_Consumer_Insights_Counts_df)

In [0]:
# Take 'Field Name' column of second file (df2) and use it as header in first file (df1)

# Drop rows where 'Field Name' column is null
field_name=Total_Consumer_Insights_Counts_df.dropna(subset=['Field Name'])

# Select only 'Field Name' column
field_name = field_name.select('Field Name')

display(field_name)

In [0]:
# File location and type 
# catalog location : 's3://consumer-orbit-us-east-1-holding/Total_Consumer_Insights_Counts (9) - Copy.xlsx'

file_location3 = 's3://consumer-orbit-us-east-1-holding/Total_Consumer_Insights_Counts (9) - Copy.xlsx'
file_type = "com.crealytics.spark.excel"

df_Insight_Plus= spark.read.format(file_type) \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .option("dataAddress", "'Sheet1'!A11") \
  .load(file_location3)

# Drop rows where all columns are null
df_Insight_Plus = df_Insight_Plus.dropna(how='all')

# Display the schema to find the correct column name
df_Insight_Plus.printSchema()


display(df_Insight_Plus)

In [0]:
# Take 'Field Name' column of second file (df2) and use it as header in first file (df1)

# Drop rows where 'Field Name' column is null
field_name2=df_Insight_Plus.dropna(subset=['Field Name'])

# Select only 'Field Name' column
field_name2 = field_name2.select('Field Name')

display(field_name2)

In [0]:
# Append field_name with field_name2 and deduplicate them
field_name_combined = field_name.union(field_name2).distinct()

display(field_name_combined)

In [0]:
# Use combined_field_name as header in df_data
combined_field_name_list = [row['Field Name'] for row in field_name_combined.collect()]
df_data = df_data.toDF(*combined_field_name_list)
display(df_data)

In [0]:
file_location4= 's3://consumer-orbit-us-east-1-holding/Alteryx Output Conversion/tci_full_import.zip'

In [0]:
df_validation = spark.read.format("csv").option("header", "true").load(f"s3:{file_location4}")